In [26]:
import ipywidgets as widgets
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from scipy.spatial import distance

In [37]:
def get_n_nearest_by_corr(data, columns, number_res=5):
    groups = []
    for c in columns:
        close_ind = data[c+' act'].sort_values(ascending=False).index
        close_res = []
        for element in close_ind:
            if element.split()[-1] == 'res':
                close_res.append(element)
        groups.append([c]+close_res[0:number_res])
    return groups


def get_n_nearest_by_dist(data, columns, number_res=5):
    groups = []
    for c in columns:
        close_ind = data[c+' act'].sort_values().index
        close_res = []
        for element in close_ind:
            if element.split()[-1] == 'res':
                close_res.append(element)
        groups.append([c]+close_res[0:number_res])
    return groups

In [21]:
df = pd.read_csv('file3_fact.csv')

df_empty_column = []
for c in df.columns:
    if (df[c] == 0).all():
        df_empty_column.append(c)


columns = [c for c in df.columns.tolist() if c not in df_empty_column]
df = df[columns]

df = df.astype(float)

In [22]:
act = [c for c in df.columns if c.split()[-1]=='act']
res = [c for c in df.columns if c.split()[-1]=='res']

In [23]:
corr = df.corr('spearman')

In [ ]:
df_normed = pd.DataFrame(data = MinMaxScaler().fit_transform(df.values), columns=df.columns)

In [ ]:
distance_df = pd.DataFrame(data=np.zeros((len(res), len(act))), columns=act)
distance_df.index = res

In [ ]:
for c1 in act:
    non_emty_act_ind = []
    for i in df.index:
        if df.loc[i,c1] != 0:
            non_emty_act_ind.append(i)
    for c2 in res:
        distance_df.loc[c2, c1] = distance.euclidean(df_normed.loc[non_emty_act_ind, c1].values, df_normed.loc[non_emty_act_ind, c2].values)


In [38]:
w1 = widgets.Dropdown(
    options=['Бурение скважин', 'Изготовление металлоконструкицй', 'Изготовление свай', 
    'Монтаж металлоконструкций', 'Монтаж оголовников свай', 'Монтаж опор', 'Надземная прокладка трубопровода',
    'Сварка трубопровода', 'Срезка свай', 'Устройство свайного основания'],
    description='Работа:',value='Бурение скважин',
    disabled=False)
w2 = widgets.IntSlider(
    value=7,
    min=0,
    max=10,
    step=1,
    description='Кол-во ближайших ресурсов:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
output = widgets.Output()
display(w1, w2, output)
def on_value_change(change):
    with output:
        print(get_n_nearest_by_corr(corr, [w1.value], w2.value))
        
     

w1.observe(on_value_change, names='value')
w2.observe(on_value_change, names='value')
#widgets.HBox([widgets.VBox([w1, w2])])

# act_selected = w1
# n_nearest = w2
# group1 = get_n_nearest_by_corr(corr, [act_selected], n_nearest)
# group2 = get_n_nearest_by_dist(distance_df, [act_selected], n_nearest)

# widgets.ToggleButtons(
#     options=[str(group1), str(group2)],
#     description='Speed:',
#     disabled=False,
#     button_style='', # 'success', 'info', 'warning', 'danger' or ''
#     tooltips=['Description of slow', 'Description of regular', 'Description of fast'],
# #     icons=['check'] * 3
# )

Dropdown(description='Работа:', options=('Бурение скважин', 'Изготовление металлоконструкицй', 'Изготовление с…

IntSlider(value=7, continuous_update=False, description='Кол-во ближайших ресурсов:', max=10)

Output()